In [19]:
import pickle
from rdkit.Chem import MolFromSmiles
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
import numpy as np
import pandas as pd
from rdkit.Chem.rdmolops import RDKFingerprint
from functools import partial
from collections import defaultdict


with open('/tf/notebooks/code_for_pub/smiles_files/smiles_drugcomb.pickle', 'rb') as f:
    sms = pickle.load(f)

In [4]:
def generate_FPs(smiles, radius=2, length=300, name='morgan'):
    '''
    smiles df should contain smiles column and cid column
    returns morganFP
    '''
    from rdkit.Chem import MolFromSmiles
    from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
    if name == 'morgan':
        f = partial(GetMorganFingerprintAsBitVect,radius=radius, nBits=length)
    else: 
        f = partial(RDKFingerprint, fpSize = length)
    holder = np.zeros((len(smiles), length), dtype=np.uint8)
    for e,i in enumerate(smiles.iteritems()):
        a = MolFromSmiles(i[1])
        #holder[e] = np.asarray(GetMorganFingerprintAsBitVect(a, radius=radius, nBits=length), dtype=np.uint8)
        holder[e] = np.asarray(f(a), dtype = np.uint8 )
        
    return holder

In [5]:
import pandas as pd
smiles = pd.DataFrame(sms)

fps_topo = generate_FPs(sms)
df = pd.DataFrame(fps_topo, index = sms.index)
fin = pd.concat((smiles, df), axis=1)
fin.shape

(4153, 301)

In [11]:
#fin.to_csv('/tf/notebooks/code_for_pub/smiles_file/fps_morga.csv', header=True, index=True)

In [11]:
#fin.to_csv('/tf/notebooks/code_for_pub/fp_files/fps_topo.csv', header=True, index=True)

In [25]:
fin.drop(columns=['smiles'], inplace=True)

In [27]:
with open('/tf/notebooks/code_for_pub/fp_files/fps_topo_300bit.pickle','wb') as f:
    pickle.dump(fin, f)

In [20]:
with open('/tf/notebooks/code_for_pub/smiles_files/smiles_drugcomb.pickle','rb') as f:
    test = pickle.load(f)